In [80]:
import gym
import numpy as np
import torch
import torch.nn as nn

from stable_baselines3 import DQN
from sb3_contrib import QRDQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat, configure
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
#from stable_baselines3.her.her_replay_buffer import HerReplayBuffer

#import minesweeper_gym

In [81]:
torch.cuda.set_device(1)

In [82]:
import sys
from six import StringIO
from random import randint

import numpy as np
import gym
from gym import spaces

# default : easy board
BOARD_SIZE = 4
NUM_MINES = 1

# cell values, non-negatives indicate number of neighboring mines
MINE = -1
CLOSED = -2


def board2str(board, end='\n'):
    """
    Format a board as a string

    Parameters
    ----
    board : np.array
    end : str

    Returns
    ----
    s : str
    """
    s = ''
    for x in range(board.shape[1]):
        for y in range(board.shape[2]):
            s += str(board[0][x][y]) + '\t'
        s += end
    return s[:-len(end)]


def is_new_move(my_board, x, y):
    """ return true if this is not an already clicked place"""
    return my_board[0, x, y] == CLOSED


def is_valid(x, y):
    """ returns if the coordinate is valid"""
    return (x >= 0) & (x < BOARD_SIZE) & (y >= 0) & (y < BOARD_SIZE)


def is_win(my_board):
    """ return if the game is won """
    return np.count_nonzero(my_board == CLOSED) == NUM_MINES


def is_mine(board, x, y):
    """return if the coordinate has a mine or not"""
    return board[0, x, y] == MINE


def place_mines(board_size, num_mines):
    """generate a board, place mines randomly"""
    mines_placed = 0
    board = np.zeros((1, board_size, board_size), dtype=int)
    while mines_placed < num_mines:
        rnd = randint(0, board_size * board_size)
        x = int(rnd / board_size)
        y = int(rnd % board_size)
        if is_valid(x, y):
            if not is_mine(board, x, y):
                board[0, x, y] = MINE
                mines_placed += 1
    return board

class MinesweeperDiscreetEnv(gym.Env):
    metadata = {"render.modes": ["ansi", "human"]}

    def __init__(self, board_size=BOARD_SIZE, num_mines=NUM_MINES):
        """
        Create a minesweeper game.

        Parameters
        ----
        board_size: int     shape of the board
            - int: the same as (int, int)
        num_mines: int   num mines on board
        """

        self.board_size = board_size
        self.num_mines = num_mines
        self.board = place_mines(board_size, num_mines)
        self.my_board = np.ones((board_size, board_size), dtype=int) * CLOSED
        self.num_actions = 0

        self.observation_space = spaces.Box(low=-2, high=9,
                                            shape=(1, self.board_size, self.board_size), dtype=np.int)
        self.action_space = spaces.Discrete(self.board_size*self.board_size)
        self.valid_actions = np.ones((self.board_size * self.board_size), dtype=np.bool)

    def count_neighbour_mines(self, x, y):
        """return number of mines in neighbour cells given an x-y coordinate

            Cell -->Current Cell(row, col)
            N -->  North(row - 1, col)
            S -->  South(row + 1, col)
            E -->  East(row, col + 1)
            W -->  West(row, col - 1)
            N.E --> North - East(row - 1, col + 1)
            N.W --> North - West(row - 1, col - 1)
            S.E --> South - East(row + 1, col + 1)
            S.W --> South - West(row + 1, col - 1)
        """
        neighbour_mines = 0
        for _x in range(x - 1, x + 2):
            for _y in range(y - 1, y + 2):
                if is_valid(_x, _y):
                    if is_mine(self.board, _x, _y):
                        neighbour_mines += 1
        return neighbour_mines

    def open_neighbour_cells(self, my_board, x, y):
        """return number of mines in neighbour cells given an x-y coordinate

            Cell -->Current Cell(row, col)
            N -->  North(row - 1, col)
            S -->  South(row + 1, col)
            E -->  East(row, col + 1)
            W -->  West(row, col - 1)
            N.E --> North - East(row - 1, col + 1)
            N.W --> North - West(row - 1, col - 1)
            S.E --> South - East(row + 1, col + 1)
            S.W --> South - West(row + 1, col - 1)
        """
        for _x in range(x-1, x+2):
            for _y in range(y-1, y+2):
                if is_valid(_x, _y):
                    if is_new_move(my_board, _x, _y):
                        my_board[0, _x, _y] = self.count_neighbour_mines(_x, _y)
                        if my_board[0, _x, _y] == 0:
                            my_board = self.open_neighbour_cells(my_board, _x, _y)
        return my_board

    def get_next_state(self, state, x, y):
        """
        Get the next state.

        Parameters
        ----
        state : (np.array)   visible board
        x : int    location
        y : int    location

        Returns
        ----
        next_state : (np.array)    next visible board
        game_over : (bool) true if game over

        """
        my_board = state
        game_over = False
        if is_mine(self.board, x, y):
            my_board[0, x, y] = MINE
            game_over = True
        else:
            my_board[0, x, y] = self.count_neighbour_mines(x, y)
            if my_board[0, x, y] == 0:
                my_board = self.open_neighbour_cells(my_board, x, y)
        self.my_board = my_board
        return my_board, game_over

    def reset(self):
        """
        Reset a new game episode. See gym.Env.reset()

        Returns
        ----
        next_state : (np.array, int)    next board
        """
        self.my_board = np.ones((1, self.board_size, self.board_size), dtype=int) * CLOSED
        self.board = place_mines(self.board_size, self.num_mines)
        self.num_actions = 0
        self.valid_actions = np.ones((self.board_size * self.board_size), dtype=bool)
        self.win_or_lose = None
        return self.my_board

    def step(self, action):
        """
        See gym.Env.step().

        Parameters
        ----
        action : np.array    location

        Returns
        ----
        next_state : (np.array)    next board
        reward : float        the reward for action
        done : bool           whether the game end or not
        info : {}             {'valid_actions': valid_actions} - a binary vector,
                                where false cells' values are already known to observer
        """
        state = self.my_board
        x = int(action / self.board_size)
        y = int(action % self.board_size)

        # test valid action - uncomment this part to test your action filter if needed
        # if bool(self.valid_actions[action]) is False:
        #    raise Exception("Invalid action was selected! Action Filter: {}, "
        #                    "action taken: {}".format(self.valid_actions, action))
        reward = 0
        while reward==0:
          next_state, reward, done, info = self.next_step(state, x, y)
        self.my_board = next_state
        self.num_actions += 1
        #self.valid_actions = (next_state.flatten() == CLOSED)
        self.valid_actions = (self.my_board == -2).reshape(self.valid_actions.shape)
        info['valid_actions'] = self.valid_actions
        info['num_actions'] = self.num_actions
        self.win_or_lose = info['is_success']
        return next_state, reward, done, info

    def is_guess(self, my_board, x, y):
        for _x in range(x-1, x+2):
            for _y in range(y-1, y+2):
                if is_valid(_x, _y):
                    if not is_new_move(my_board, _x, _y):
                        if (x != _x) or (y != _y):
                            return False
        return True
                    

    def next_step(self, state, x, y):
        """
        Get the next observation, reward, done, and info.

        Parameters
        ----
        state : (np.array)    visible board
        x : int    location
        y : int    location

        Returns
        ----
        next_state : (np.array)    next visible board
        reward : float               the reward
        done : bool           whether the game end or not
        info : {}
        """
        my_board = state
        win_or_lose = False
        reward = 0
        done = False
        t_b = False
        if not is_new_move(my_board, x, y):
            return my_board, -0.3, False, {'is_success': win_or_lose}
        elif self.is_guess(my_board, x, y): # if guess
            t_b = True

        state, game_over = self.get_next_state(my_board, x, y)
        if game_over:
            reward = -1
            done = True
            #return state, -100, True, {}
        elif is_win(state):
            reward = 1
            done = True
            win_or_lose = True
            #return state, 1000, True, {}
        elif t_b: # if guess
            # random x, y
            reward = -0.3
        else: # progress
            reward = 0.9
            #return state, 0, False, {}
            
        return state, reward, done, {'is_success': win_or_lose}

    def render(self, mode='human'):
        """
        See gym.Env.render().
        """
        outfile = StringIO() if mode == 'ansi' else sys.stdout
        s = board2str(self.my_board)
        outfile.write(s)
        if mode != 'human':
            return outfile

In [83]:
def evaluate(model, num_steps=1000):
  """
  Evaluate a RL agent
  :param model: (BaseRLModel object) the RL Agent
  :param num_steps: (int) number of timesteps to evaluate it
  :return: (float) Mean reward for the last 100 episodes
  """
  episode_rewards = [0.0]
  obs = env.reset()
  n_games = 0
  n_win = 0
  
  for i in range(num_steps):
      # _states are only useful when using LSTM policies
      action, _states = model.predict(obs)

      obs, reward, done, info = env.step(action)
      
      # Stats
      episode_rewards[-1] += reward
      if done:
          n_games += 1
          if env.win_or_lose:
              n_win += 1
          obs = env.reset()
          episode_rewards.append(0.0)
  # Compute mean reward for the last 100 episodes
  mean_100ep_reward = round(np.mean(episode_rewards[-100:]), 1)
  print("Mean reward:", mean_100ep_reward, "Num episodes:", len(episode_rewards), "Win rate:", round(n_win/n_games, 3))
  
  return mean_100ep_reward

In [ ]:
nn.Conv2d(num_channels, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            Flatten(),

In [107]:
class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 512):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # We assume CxHxW images (channels first)
        # Re-ordering will be done by pre-preprocessing or wrapper
        n_input_channels = observation_space.sample()[None].shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding='same', bias=True),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding='same', bias=True),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding='same', bias=True),
            nn.ReLU(),
#             nn.Conv2d(128, 128, kernel_size=3, stride=1, padding='same'),
#             nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with torch.no_grad():
            n_flatten = self.cnn(
                torch.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(
            nn.Linear(n_flatten, features_dim),
            nn.ReLU(),
            nn.Linear(features_dim, features_dim),
            nn.ReLU()
        )


    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))

In [108]:
def exp_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    lr0 = initial_value
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining: = 1.0 - (num_timesteps / total_timesteps)
        :return: current learning rate
        """
        #if progress_remaining > 0.5:
        #    return initial_value
        #else:
        #    return progress_remaining * initial_value * 2
        #return progress_remaining * initial_value
        nonlocal lr0
        lr0 = max(0.001, lr0 * 0.99975) # 0.99975
        return lr0

    return func

In [109]:
from typing import Callable
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining: 1 - num_step / total_step
        :return: current learning rate
        """
        if progress_remaining > 0.5:
            return initial_value
        else:
            return progress_remaining * initial_value *2

    return func

In [114]:
env = MinesweeperDiscreetEnv(4,1)
policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=512),
)
model = QRDQN('CnnPolicy', env, 
            #learning_rate=1e-3, 
            #policy_kwargs=dict(activation_fn=nn.ReLU,
            #                   net_arch=[256, 256, 256]), 
            policy_kwargs=policy_kwargs,
            learning_rate=linear_schedule(0.001),
            #batch_size=1, 
            gamma=0.1, 
            #tau=0.8,
            train_freq=(5, 'episode'),
            target_update_interval=100,
            learning_starts=1,
            #buffer_size=4,
            exploration_fraction=0.9, 
            exploration_initial_eps=0.95, 
            exploration_final_eps=0.01,
            tensorboard_log="./qrdqn_tensorboard/", verbose=0)

/tmp/ipykernel_515384/2364046332.py:94: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  shape=(1, self.board_size, self.board_size), dtype=np.int)
/tmp/ipykernel_515384/2364046332.py:96: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecation

In [115]:
#callback = TensorboardCallback(eval_env=MinesweeperDiscreetEnv())
model.learn(total_timesteps=int(2e5), log_interval=10, reset_num_timesteps=True, tb_log_name = 'QRDQN_s4m1')

In [116]:
model.save("QRDQN_s4m1")